In [1]:
import sys

# add project root directory to python path to enable import of saapy
if ".." not in sys.path:
    sys.path.append('..')

In [2]:
from saapy.workspace import Workspace
ws = Workspace.from_home("iDempiere", rel_root_dir="Projects", work_dir="iDempiere")

In [3]:
import logging
logging.captureWarnings(True) # mostly warnings caused by self-signed certs

In [4]:
# This does not do much at the moment, mostly test of connectivity to sonar

from saapy.clients import SonarClient

sonar_url = ws.get_service_url("sonar_decomposeit")
sonar_token_name, sonar_token = ws.get_service_credentials("sonar_decomposeit")
sonar = SonarClient(sonar_url, sonar_token)
sonar_dups = sonar.get("api/duplications/show", 
                       dict(key="IdempiereKey:temp/org.adempiere.extend/src/test/functional/MBPGroupTest.java"))